In [1]:
from google.colab import files
uploaded = files.upload()

Saving tmp.txt to tmp.txt


In [0]:
import pandas as pd

In [0]:
data = uploaded['tmp.txt'].decode('ascii')

weights = []
lengths = []
for x in data.split('\n')[1:]:
    weights.append(int(x.split()[0]))
    lengths.append(int(x.split()[1]))

In [0]:
df = pd.DataFrame({"weights": weights, "lengths": lengths})

In [0]:
df["difference"] = df["weights"] - df["lengths"]

In [31]:
df.sort_values(["difference", "weights"], ascending=[False, False], inplace=True)
df["completion_times"] = df["lengths"].cumsum()
df["weighted_completion_times"] = df["weights"] * df["completion_times"]
df["weighted_completion_times"].sum()

69119377652

In [0]:
df["ratio"] = df["weights"] / df["lengths"]

In [30]:
df.sort_values(["ratio", "weights"], ascending=[False, False], inplace=True)
df["completion_times"] = df["lengths"].cumsum()
df["weighted_completion_times"] = df["weights"] * df["completion_times"]
df["weighted_completion_times"].sum()

67311454237

In [32]:
from google.colab import files
uploaded_edges = files.upload()

Saving edges.txt to edges.txt


In [90]:
uploaded_edges = open('/Users/nicolas/edges.txt', 'r')
data = uploaded_edges.read().split('\n')

In [73]:
data = data[:-1]

In [91]:
n_nodes = int(data[0].split()[0])
n_edges = int(data[0].split()[1])

edges = []
costs = []
in_edges = []
out_edges = []
for line in data[1:]:
    u, v, cost = [int(x) for x in line.split()]
    edges.append((u, v))
    in_edges.append(u)
    out_edges.append(v)
    costs.append(cost)

In [92]:
import pandas as pd

df = pd.DataFrame({"in_edges": in_edges, "out_edges": out_edges, "costs": costs})
df.head()

,in_edges,out_edges,costs
0,1,2,6807
1,2,3,-8874
2,3,4,-1055
3,4,5,4414
4,5,6,1728


In [93]:
# Naive implementation in O(n*m)
import random
random_node = 4 #random.randint(1, n_nodes)
X = {random_node}  # choose a random vertex (a.k.a node)
T = set() # list of MST edges
remaining_nodes = set(range(1, n_nodes+1))
remaining_nodes.remove(random_node)
print("Random node: {}".format(random_node))

i = 0
while X != set(range(1, n_nodes+1)):
#     crossing_edges = [i for i, edge in enumerate(edges) 
#                       if edge[0] in X and edge[1] in remaining_nodes]
    crossing_edges = df[(df["in_edges"].isin(X) & df["out_edges"].isin(remaining_nodes))
                        | (df["in_edges"].isin(remaining_nodes) & df["out_edges"].isin(X))]

#     print(crossing_edges)
    min_edge = crossing_edges.loc[crossing_edges["costs"].idxmin()] # cheapest edge with u in X and v not in X
    new_edge = "in_edges" if min_edge["out_edges"] in X else "out_edges"
    T.add(min_edge.name)
    X.add(min_edge[new_edge])
    remaining_nodes.remove(min_edge[new_edge])
    i += 1
#     print(i)
#     print(X)
#     print(remaining_nodes)
#     print("Time: {}".format(time.time()-t0))
#     print('='*30)

Random node: 4


In [94]:
df.loc[T]["costs"].sum()

-3612829